# Deep Autoencoders

#### by Khaled Nasr as a part of a <a href="https://www.google-melange.com/gsoc/project/details/google/gsoc2014/khalednasr92/5657382461898752">GSoC 2014 project</a> mentored by Theofanis Karaletsos and Sergey Lisitsyn 

This notebook illustrates how to train and evaluate a deep autoencoder using Shogun. We'll look at both regular fully-connected autoencoders and convolutional autoencoders.

## Introduction

A (single layer) [autoencoder](http://deeplearning.net/tutorial/dA.html#autoencoders) is a neural network that has three layers: an input layer, a hidden (encoding) layer, and a decoding layer. The network is trained to reconstruct its inputs, which forces the hidden layer to try to learn good representations of the inputs.

In order to encourage the hidden layer to learn good input representations, certain variations on the simple autoencoder exist. Shogun currently supports two of them: Denoising Autoencoders [1] and Contractive Autoencoders [2]. In this notebook we'll focus on denoising autoencoders. 

For denoising autoencoders, each time a new training example is introduced to the network, it's randomly corrupted in some mannar, and the target is set to the original example. The autoencoder will try to recover the orignal data from it's noisy version, which is why it's called a denoising autoencoder. This process will force the hidden layer to learn a good representation of the input, one which is not affected by the corruption process.

A deep autoencoder is an autoencoder with multiple hidden layers. Training such autoencoders directly is usually difficult, however, they can be pre-trained as a stack of single layer autoencoders. That is, we train the first hidden layer to reconstruct the input data, and then train the second hidden layer to reconstruct the states of the first hidden layer, and so on. After pre-training, we can train the entire deep autoencoder to fine-tune all the parameters together. We can also use the autoencoder to initialize a regular neural network and train it in a supervised manner.

In this notebook we'll apply deep autoencoders to the USPS dataset for handwritten digits. We'll start by loading the data and dividing it into a training set and a test set:

In [ ]:
import os
SHOGUN_DATA_DIR=os.getenv('SHOGUN_DATA_DIR', '../../../data')
from scipy.io import loadmat
import shogun as sg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# load the dataset
dataset = loadmat(os.path.join(SHOGUN_DATA_DIR, 'multiclass/usps.mat'))

Xall = dataset['data']
# the usps dataset has the digits labeled from 1 to 10 
# we'll subtract 1 to make them in the 0-9 range instead
Yall = np.array(dataset['label'].squeeze(), dtype=np.double)-1 

# 4000 examples for training
Xtrain = sg.create_features(Xall[:,0:4000])
Ytrain = sg.create_labels(Yall[0:4000])

# the rest for testing
Xtest = sg.create_features(Xall[:,4000:-1])
Ytest = sg.create_labels(Yall[4000:-1])

## Creating the autoencoder

Similar to regular neural networks in Shogun, we create a [deep autoencoder](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html) using an array of [NeuralLayer](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1NeuralLayer.html)-based classes, which can be created using the utility class [NeuralLayers](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1NeuralLayers.html). However, for deep autoencoders there's a restriction that the layer sizes in the network have to be symmetric, that is, the first layer has to have the same size as the last layer, the second layer has to have the same size as the second-to-last layer, and so on. This restriction is necessary for pre-training to work. More details on that can found in the following section.

We'll create a 5-layer deep autoencoder with following layer sizes: 256->512->128->512->256. We'll use [rectified linear neurons](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1NeuralRectifiedLinearLayer.html) for the hidden layers and [linear neurons](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1NeuralLinearLayer.html) for the output layer.

In [ ]:
ae = sg.create_machine("DeepAutoencoder", seed=10)
ae.add("layers", sg.create_layer("NeuralInputLayer", num_neurons=256))
ae.add("layers", sg.create_layer("NeuralRectifiedLinearLayer", num_neurons=512))
ae.add("layers", sg.create_layer("NeuralRectifiedLinearLayer", num_neurons=128))
ae.add("layers", sg.create_layer("NeuralRectifiedLinearLayer", num_neurons=512))
ae.add("layers", sg.create_layer("NeuralLinearLayer", num_neurons=256))

## Pre-training

Now we can pre-train the network. To illustrate exactly what's going to happen, we'll give the layers some labels: L1 for the input layer, L2 for the first hidden layer, and so on up to L5 for the output layer.

In pre-training, an autoencoder will formed for each encoding layer (layers up to the middle layer in the network). So here we'll have two autoencoders: L1->L2->L5, and L2->L3->L4. The first autoencoder will be trained on the raw data and used to initialize the weights and biases of layers L2 and L5 in the deep autoencoder. After the first autoencoder is trained, we use it to transform the raw data into the states of L2. These states will then be used to train the second autoencoder, which will be used to initialize the weights and biases of layers L3 and L4 in the deep autoencoder.

The operations described above are performed by the the [pre_train()](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html#acf6896cb166afbba063fd1257cb8bc97) function. Pre-training parameters for each autoencoder can be controlled using the [pt_* public attributes](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html#a6389a6f19b8854c64e1b6be5aa0c1fc4) of [DeepAutoencoder](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html). Each of those attributes is an [SGVector](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1SGVector.html) whose length is the number of autoencoders in the deep autoencoder (2 in our case). It can be used to set the parameters for each autoencoder indiviually. [SGVector's set_const()](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1SGVector.html#a8bce01a1fc41a734d9b5cf1533fd7a2a) method can also be used to assign the same parameter value for all autoencoders.

Different noise types can be used to corrupt the inputs in a denoising autoencoder. Shogun currently supports 2 [noise types](http://www.shogun-toolbox.org/doc/en/latest/namespaceshogun.html#af95cf5d3778127a87c8a67516405d863): dropout noise, where a random portion of the inputs is set to zero at each iteration in training, and gaussian noise, where the inputs are corrupted with random gaussian noise. The noise type and strength can be controlled using [pt_noise_type](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html#af6e5d2ade5cb270cc50565d590f929ae) and [pt_noise_parameter](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html#adbdff6c07fa7dd70aaf547e192365075). Here, we'll use dropout noise.

In [ ]:
ae.put("noise_type", "AENT_DROPOUT") # use dropout noise
ae.put("noise_parameter", 0.5) # each input has a 50% chance of being set to zero

ae.put("optimization_method", "NNOM_GRADIENT_DESCENT") # train using gradient descent
ae.put("gd_learning_rate", 0.01)
ae.put("gd_mini_batch_size", 128)

ae.put("max_num_epochs", 50)
ae.put("epsilon", 0.0) # disable automatic convergence testing

# uncomment this line to allow the training progress to be printed on the console
# from shogun import MSG_INFO; sg.env().set_loglevel(MSG_INFO)

# tell ae to do pre-training.
ae.put("do_pretrain", True)

## Fine-tuning

After pre-training, we can train the autoencoder as a whole to fine-tune the parameters. Training the whole autoencoder is performed using the [train()](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1Autoencoder.html#ace3eb6cc545affcbfa31d754ffd087dc) function. Training parameters are controlled through the [public attributes](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html#pub-attribs), same as a regular neural network.

In [ ]:
ae.put('noise_type', "AENT_DROPOUT") # same noise type we used for pre-training
ae.put('noise_parameter', 0.5)

ae.put('max_num_epochs', 50)
ae.put('optimization_method', "NNOM_GRADIENT_DESCENT")
ae.put('gd_mini_batch_size', 128)
ae.put('gd_learning_rate', 0.0001)
ae.put('epsilon', 0.0)

# start fine-tuning. this might take some time
_ = ae.train(Xtrain)

## Evaluation

Now we can evaluate the autoencoder that we trained. We'll start by providing it with corrupted inputs and looking at how it will reconstruct them. The function [reconstruct()](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html#ae8c2d565cf2ea809103d0557c57689c7) is used to obtain the reconstructions:

In [ ]:
# get a 50-example subset of the test set
subset = Xtest.get("feature_matrix")[:,0:50]

# corrupt the first 25 examples with multiplicative noise
subset[:,0:25] *= (np.random.random((256,25))>0.5)

# corrupt the other 25 examples with additive noise 
subset[:,25:50] += np.random.random((256,25))

# obtain the reconstructions
reconstructed_subset = sg.reconstruct(ae, sg.create_features(subset))

# plot the corrupted data and the reconstructions
plt.figure(figsize=(10,10))
for i in range(50):
    ax1=plt.subplot(10,10,i*2+1)
    ax1.imshow(subset[:,i].reshape((16,16)), interpolation='nearest', cmap = matplotlib.cm.Greys_r)
    ax1.set_xticks([])
    ax1.set_yticks([])

    ax2=plt.subplot(10,10,i*2+2)
    ax2.imshow(reconstructed_subset.get("feature_matrix")[:,i].reshape((16,16)), 
               interpolation='nearest', cmap = matplotlib.cm.Greys_r)
    ax2.set_xticks([])
    ax2.set_yticks([])

The figure shows the corrupted examples and their reconstructions. The top half of the figure shows the ones corrupted with multiplicative noise, the bottom half shows the ones corrupted with additive noise. We can see that the autoencoders can provide decent reconstructions despite the heavy noise.

Next we'll look at the weights that the first hidden layer has learned. To obtain the weights, we can call the [get_layer_parameters()]() function, which will return a vector containing both the weights and the biases of the layer. The biases are stored first in the array followed by the weights matrix in column-major format.

In [ ]:
# obtain the weights matrix of the first hidden layer
# the 512 is the number of biases in the layer (512 neurons)
# the transpose is because numpy stores matrices in row-major format, and Shogun stores 
# them in column major format
w1 = ae.get("layer_parameters")[0][512:].reshape(256,512).T

# visualize the weights between the first 100 neurons in the hidden layer 
# and the neurons in the input layer
plt.figure(figsize=(10,10))
for i in range(100):
	ax1=plt.subplot(10,10,i+1)
	ax1.imshow(w1[i,:].reshape((16,16)), interpolation='nearest', cmap = matplotlib.cm.Greys_r)
	ax1.set_xticks([])
	ax1.set_yticks([])

Now, we can use the autoencoder to initialize a supervised neural network. The network will have all the layer of the autoencoder up to (and including) the middle layer. We'll also add a softmax output layer. So, the network will look like: L1->L2->L3->Softmax. The network is obtained by calling [convert_to_neural_network()](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DeepAutoencoder.html#a8c179cd9a503b2fa78b9bfe10ae473e5):

In [ ]:
nn = sg.convert_to_neural_network(ae, sg.create_layer("NeuralSoftmaxLayer", num_neurons=10), 0.01)

nn.put('max_num_epochs', 50)

_ = nn.train(Xtrain, Ytrain)

Next, we'll evaluate the accuracy on the test set:

In [ ]:
predictions = nn.apply(Xtest)
accuracy = sg.create_evaluation("MulticlassAccuracy").evaluate(predictions, Ytest) * 100

print("Classification accuracy on the test set =", accuracy, "%")

## Convolutional Autoencoders

Convolutional autoencoders [3] are the adaptation of autoencoders to images (or other spacially-structured data). They are built with convolutional layers where each layer consists of a number of feature maps. Each feature map is produced by convolving a small filter with the layer's inputs, adding a bias, and then applying some non-linear activation function. Additionally, a max-pooling operation can be performed on each feature map by dividing it into small non-overlapping regions and taking the maximum over each region. In this section we'll pre-train a [convolutional network](http://deeplearning.net/tutorial/lenet.html) as a stacked autoencoder and use it for classification.

In Shogun, convolutional autoencoders are constructed and trained just like regular autoencoders. Except that we build the autoencoder using [NeuralConvolutionalLayer](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1NeuralConvolutionalLayer.html) objects:

In [ ]:
conv_ae = sg.create_machine("DeepAutoencoder", seed=10)
# 16x16 single channel images
conv_ae.add("layers", sg.create_layer("NeuralInputLayer", width=16, height=16, num_neurons=256)) 

# the first encoding layer: 5 feature maps, filters with radius 2 (5x5 filters)
# and max-pooling in a 2x2 region: its output will be 10 8x8 feature maps
conv_ae.add("layers", sg.create_layer("NeuralConvolutionalLayer",
                               activation_function="CMAF_RECTIFIED_LINEAR", 
                               num_maps=5, 
                               radius_x=2, 
                               radius_y=2, 
                               pooling_width=2, 
                               pooling_height=2)) 

# the second encoding layer: 15 feature maps, filters with radius 2 (5x5 filters)
# and max-pooling in a 2x2 region: its output will be 20 4x4 feature maps
conv_ae.add("layers", sg.create_layer("NeuralConvolutionalLayer",
                               activation_function="CMAF_RECTIFIED_LINEAR", 
                               num_maps=15, 
                               radius_x=2, 
                               radius_y=2, 
                               pooling_width=2, 
                               pooling_height=2)) 

# the first decoding layer: same structure as the first encoding layer
conv_ae.add("layers", sg.create_layer("NeuralConvolutionalLayer",
                               activation_function="CMAF_RECTIFIED_LINEAR", 
                               num_maps=15, 
                               radius_x=2, 
                               radius_y=2))

# the second decoding layer: same structure as the input layer
conv_ae.add("layers", sg.create_layer("NeuralConvolutionalLayer",
                               activation_function="CMAF_RECTIFIED_LINEAR", 
                               num_maps=1, 
                               radius_x=2, 
                               radius_y=2))

Now we'll pre-train the autoencoder:

In [ ]:
ae.put("noise_type", "AENT_DROPOUT") # use dropout noise
ae.put("noise_parameter", 0.3) # each input has a 50% chance of being set to zero

ae.put("optimization_method", "NNOM_GRADIENT_DESCENT") # train using gradient descent
ae.put("gd_learning_rate", 0.002)
ae.put("gd_mini_batch_size", 100)

ae.put("max_num_epochs", 30)

# tell ae to do pre-training.
ae.put("do_pretrain", True)

And then convert the autoencoder to a regular neural network for classification:

In [ ]:
conv_nn = sg.convert_to_neural_network(ae, sg.create_layer("NeuralSoftmaxLayer", num_neurons=10), 0.01)

# train the network
conv_nn.put('epsilon', 0.0)
conv_nn.put('max_num_epochs', 50)

# start training. this might take some time
_ = conv_nn.train(Xtrain, Ytrain)

And evaluate it on the test set:

In [ ]:
predictions = conv_nn.apply_multiclass(Xtest)
accuracy = sg.create_evaluation("MulticlassAccuracy").evaluate(predictions, Ytest) * 100

print("Classification accuracy on the test set =", accuracy, "%")

## References

- [1] [Stacked Denoising Autoencoders: Learning Useful Representations in a Deep Network with a Local Denoising Criterion, Vincent, 2010](http://jmlr.org/papers/volume11/vincent10a/vincent10a.pdf)
- [2] [Contractive Auto-Encoders: Explicit Invariance During Feature Extraction, Rifai, 2011](http://machinelearning.wustl.edu/mlpapers/paper_files/ICML2011Rifai_455.pdf)
- [3] [Stacked Convolutional Auto-Encoders for Hierarchical Feature Extraction, J. Masci, 2011](http://www.idsia.ch/~ciresan/data/icann2011.pdf)